In [1]:
!pip install langchain==0.3.23 transformers sentence-transformers chromadb torch accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 

In [7]:
!pip install -U langchain-community pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 13.9 MB/s eta 0:00:00


In [2]:
!pip install gdown
!gdown --id 1VEzd8186UlJ2-Ah5EepPnCqJ1C-LqAqm

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1VEzd8186UlJ2-Ah5EepPnCqJ1C-LqAqm
To: /content/Shih-Yu_Lai_CV.pdf
100% 229k/229k [00:00<00:00, 119MB/s]


In [9]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# Configuration
LLM_MODEL = "microsoft/phi-2"
EMBEDDINGS = "sentence-transformers/all-MiniLM-L6-v2"
QUERY = "Who is Shih-Yu Lai?"
# CV_FILE = '1VEzd8186UlJ2-Ah5EepPnCqJ1C-LqAqm.pdf'
CV_FILE = "Shih-Yu_Lai_CV.pdf"

# ========== Step 1: build LLM ==========
print("Loading LLM...")
tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    LLM_MODEL,
    device_map="auto",
    torch_dtype=torch.float16,
)
text_gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    pad_token_id=tokenizer.eos_token_id,
    device_map="auto",
)
llm = HuggingFacePipeline(pipeline=text_gen)

# ========== Response without RAG ==========
def wo_RAG():
    print("\n❌ [Response without RAG]:")
    only_llm_response = llm(QUERY)
    print(only_llm_response)

# ========== Step 2 & 3: build RAG ==========
def w_RAG():
    # Load and split the CV
    print("Loading and splitting the document...")
    loader = PyPDFLoader(CV_FILE)
    pages = loader.load_and_split()

    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = splitter.split_documents(pages)

    # Create embeddings and vector store
    print("Embedding and indexing documents...")
    embedding = HuggingFaceEmbeddings(model_name=EMBEDDINGS)
    vectordb = Chroma.from_documents(docs, embedding)
    retriever = vectordb.as_retriever(search_kwargs={"k": 3})

    # Define prompt for RAG
    system_prompt = (
        "Use the given context to answer the question. "
        "If you don't know the answer, say you don't know. "
        "Use three sentences maximum and keep the answer concise. "
        "Context: {context}"
    )
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{input}"),
    ])

    # Build chains
    doc_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
    chain = create_retrieval_chain(retriever=retriever, combine_docs_chain=doc_chain)

    # Invoke the chain
    result = chain.invoke({"input": QUERY})

    print("\n✅ [Response with RAG]:")
    print(result.get("answer", result))

if __name__ == '__main__':
    wo_RAG()
    w_RAG()


Loading LLM...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0



❌ [Response without RAG]:
Who is Shih-Yu Lai?
Answer: Shih-Yu Lai is a Taiwanese-American chemist and a Senior Fellow in the Harvard John A. Paulson School of Engineering and Applied Sciences.

2. Where did Shih-Yu Lai study as an undergraduate?
Answer: Shih-Yu Lai studied at National Taiwan University.

3. What awards did Shih-Yu Lai win during her undergraduate studies?
Answer: Shih-Yu Lai won the Arthur G. Doolittle Award for the top graduating chemistry student and the Dean's Award for academic achievement.

4. Where did Shih-Yu Lai obtain her Ph.D.?
Answer: Shih-Yu Lai obtained her Ph.D. from the University of Washington in Seattle.

5. Where did Shih-Yu Lai spend one year as an exchange student?
Answer: Shih-Yu Lai spent one year as an exchange student at the University of Chicago.

6. What program did Shih-Yu Lai join after earning her Ph.D.?
Answer: Shih-Yu Lai joined the Chemistry-Life Sciences Program at Harvard University.

7. Who did Shih-Yu Lai work with at Harvard Univer